<a href="https://colab.research.google.com/github/SahajTC/Augmented-Financial-Intelligence/blob/main/LSTMipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt

In [2]:
pip install yfinance prophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 9.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score,mean_absolute_percentage_error 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

In [5]:
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [40]:
import yfinance as yf
btc=yf.download(tickers='BTC-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#bitcoin
eth=yf.download(tickers='ETH-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#etherum
bnb=yf.download(tickers='BNB-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#binance

aapl = yf.download(tickers='AAPL', start='2020-01-01', end='2022-11-30', interval = '1d')#apple
tsla=yf.download(tickers='TSLA', start='2020-01-01', end='2022-11-30', interval = '1d')#tesla
googl=yf.download(tickers='GOOGL', start='2020-01-01', end='2022-11-30', interval = '1d')#google

doge=yf.download(tickers='DOGE-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#dogecoin
dot=yf.download(tickers='DOT-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#polkadot
atom=yf.download(tickers='ATOM-USD', start='2020-01-01', end='2022-11-30', interval = '1d')#cosmos

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [20]:
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]    
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [37]:
def Model_fit(bth):
 bth.reset_index(inplace=True)
 bth['Date'] = pd.to_datetime(bth['Date'],format='%Y-%m-%d')
 closebth = bth[['Date','Close']]
 closebth = closebth[closebth['Date'] > '2020-01-01']

 del closebth['Date']
 scaler=MinMaxScaler(feature_range=(0,1))
 closebth=scaler.fit_transform(np.array(closebth).reshape(-1,1))

 training_size=int(len(closebth)*0.80)
 test_size=len(closebth)-training_size
 train_data,test_data=closebth[0:training_size,:],closebth[training_size:len(closebth),:1]

 time_step = 15
 X_train, y_train = create_dataset(train_data, time_step)
 X_test, y_test = create_dataset(test_data, time_step)
 X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
 X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

 model=Sequential()
 model.add(LSTM(10,input_shape=(None,1),activation="relu"))
 model.add(Dense(1))
 model.compile(loss="mean_squared_error",optimizer="adam")

 history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

 train_predict=model.predict(X_train)
 test_predict=model.predict(X_test)

 train_predict = scaler.inverse_transform(train_predict)
 test_predict = scaler.inverse_transform(test_predict)
 original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
 original_ytest = scaler.inverse_transform(y_test.reshape(-1,1))

 print("Train data explained MAPE score:", 
      mean_absolute_percentage_error(original_ytrain, train_predict))
 print("Test data explained MAPE score:", 
      mean_absolute_percentage_error(original_ytest, test_predict))

In [41]:
Model_fit(eth)

Epoch 1/200
27/27 [==============================] - 2s 17ms/step - loss: 0.0911 - val_loss: 0.0206
Epoch 2/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0271 - val_loss: 0.0019
Epoch 3/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0058 - val_loss: 0.0022
Epoch 4/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0040 - val_loss: 9.7037e-04
Epoch 5/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0033 - val_loss: 8.5861e-04
Epoch 6/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0029 - val_loss: 0.0011
Epoch 7/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0027 - val_loss: 0.0011
Epoch 8/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0026 - val_loss: 0.0018
Epoch 9/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0023 - val_loss: 0.0018
Epoch 10/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0021 - val_loss: 0.0019


In [38]:
Model_fit(btc)

Epoch 1/200
27/27 [==============================] - 2s 23ms/step - loss: 0.4333 - val_loss: 0.0818
Epoch 2/200
27/27 [==============================] - 0s 8ms/step - loss: 0.3152 - val_loss: 0.0401
Epoch 3/200
27/27 [==============================] - 0s 8ms/step - loss: 0.2147 - val_loss: 0.0158
Epoch 4/200
27/27 [==============================] - 0s 8ms/step - loss: 0.1290 - val_loss: 0.0033
Epoch 5/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0552 - val_loss: 7.8652e-04
Epoch 6/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0088 - val_loss: 8.8329e-04
Epoch 7/200
27/27 [==============================] - 0s 7ms/step - loss: 0.0011 - val_loss: 4.7399e-04
Epoch 8/200
27/27 [==============================] - 0s 8ms/step - loss: 9.8076e-04 - val_loss: 3.5796e-04
Epoch 9/200
27/27 [==============================] - 0s 8ms/step - loss: 8.8743e-04 - val_loss: 3.5441e-04
Epoch 10/200
27/27 [==============================] - 0s 8ms/step - loss: 8.5722

In [ ]:
Model_fit(tsla)

KeyError: ignored

In [ ]:
Model_fit(aapl)

Epoch 1/200
18/18 [==============================] - 2s 25ms/step - loss: 0.0258 - val_loss: 0.0036
Epoch 2/200
18/18 [==============================] - 0s 10ms/step - loss: 0.0035 - val_loss: 0.0041
Epoch 3/200
18/18 [==============================] - 0s 11ms/step - loss: 0.0027 - val_loss: 0.0027
Epoch 4/200
18/18 [==============================] - 0s 11ms/step - loss: 0.0022 - val_loss: 0.0024
Epoch 5/200
18/18 [==============================] - 0s 12ms/step - loss: 0.0019 - val_loss: 0.0023
Epoch 6/200
18/18 [==============================] - 0s 11ms/step - loss: 0.0017 - val_loss: 0.0022
Epoch 7/200
18/18 [==============================] - 0s 11ms/step - loss: 0.0016 - val_loss: 0.0021
Epoch 8/200
18/18 [==============================] - 0s 12ms/step - loss: 0.0014 - val_loss: 0.0020
Epoch 9/200
18/18 [==============================] - 0s 12ms/step - loss: 0.0013 - val_loss: 0.0020
Epoch 10/200
18/18 [==============================] - 0s 12ms/step - loss: 0.0013 - val_loss: 0.0019

In [ ]:
Model_fit(googl)

Epoch 1/200
18/18 [==============================] - 2s 23ms/step - loss: 0.7029 - val_loss: 0.4852
Epoch 2/200
18/18 [==============================] - 0s 10ms/step - loss: 0.5261 - val_loss: 0.3657
Epoch 3/200
18/18 [==============================] - 0s 9ms/step - loss: 0.3960 - val_loss: 0.2477
Epoch 4/200
18/18 [==============================] - 0s 9ms/step - loss: 0.2773 - val_loss: 0.1625
Epoch 5/200
18/18 [==============================] - 0s 9ms/step - loss: 0.1722 - val_loss: 0.0729
Epoch 6/200
18/18 [==============================] - 0s 9ms/step - loss: 0.0508 - val_loss: 0.0031
Epoch 7/200
18/18 [==============================] - 0s 9ms/step - loss: 0.0086 - val_loss: 0.0072
Epoch 8/200
18/18 [==============================] - 0s 9ms/step - loss: 0.0063 - val_loss: 0.0053
Epoch 9/200
18/18 [==============================] - 0s 9ms/step - loss: 0.0058 - val_loss: 0.0055
Epoch 10/200
18/18 [==============================] - 0s 10ms/step - loss: 0.0052 - val_loss: 0.0039
Epoch 

In [42]:
Model_fit(bnb)

Epoch 1/200
27/27 [==============================] - 2s 19ms/step - loss: 0.2350 - val_loss: 0.1118
Epoch 2/200
27/27 [==============================] - 0s 8ms/step - loss: 0.1167 - val_loss: 0.0259
Epoch 3/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0224 - val_loss: 8.5659e-04
Epoch 4/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0067 - val_loss: 0.0038
Epoch 5/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0048 - val_loss: 0.0025
Epoch 6/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0038 - val_loss: 0.0014
Epoch 7/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0034 - val_loss: 0.0012
Epoch 8/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0032 - val_loss: 9.7973e-04
Epoch 9/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0028 - val_loss: 9.0515e-04
Epoch 10/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0027 - val_loss: 7.5

In [43]:
Model_fit(doge)

Epoch 1/200
27/27 [==============================] - 2s 17ms/step - loss: 0.0547 - val_loss: 0.0035
Epoch 2/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0385 - val_loss: 6.3127e-04
Epoch 3/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0293 - val_loss: 0.0010
Epoch 4/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0235 - val_loss: 0.0030
Epoch 5/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0196 - val_loss: 0.0041
Epoch 6/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0158 - val_loss: 0.0040
Epoch 7/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0119 - val_loss: 0.0028
Epoch 8/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0085 - val_loss: 0.0013
Epoch 9/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0073 - val_loss: 5.3023e-04
Epoch 10/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0063 - val_loss: 3.2760

In [44]:
Model_fit(dot)

Epoch 1/200
21/21 [==============================] - 2s 20ms/step - loss: 0.1406 - val_loss: 2.7779e-04
Epoch 2/200
21/21 [==============================] - 0s 8ms/step - loss: 0.0700 - val_loss: 0.0060
Epoch 3/200
21/21 [==============================] - 0s 8ms/step - loss: 0.0243 - val_loss: 0.0220
Epoch 4/200
21/21 [==============================] - 0s 11ms/step - loss: 0.0117 - val_loss: 0.0178
Epoch 5/200
21/21 [==============================] - 0s 8ms/step - loss: 0.0080 - val_loss: 0.0087
Epoch 6/200
21/21 [==============================] - 0s 8ms/step - loss: 0.0052 - val_loss: 0.0039
Epoch 7/200
21/21 [==============================] - 0s 9ms/step - loss: 0.0037 - val_loss: 5.1945e-04
Epoch 8/200
21/21 [==============================] - 0s 9ms/step - loss: 0.0032 - val_loss: 1.5871e-04
Epoch 9/200
21/21 [==============================] - 0s 9ms/step - loss: 0.0029 - val_loss: 6.7710e-05
Epoch 10/200
21/21 [==============================] - 0s 8ms/step - loss: 0.0028 - val_loss

In [45]:
Model_fit(atom)

Epoch 1/200
27/27 [==============================] - 2s 16ms/step - loss: 0.1921 - val_loss: 0.0347
Epoch 2/200
27/27 [==============================] - 0s 8ms/step - loss: 0.1366 - val_loss: 0.0083
Epoch 3/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0714 - val_loss: 0.0043
Epoch 4/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0301 - val_loss: 0.0147
Epoch 5/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0163 - val_loss: 0.0026
Epoch 6/200
27/27 [==============================] - 0s 7ms/step - loss: 0.0089 - val_loss: 9.7375e-04
Epoch 7/200
27/27 [==============================] - 0s 9ms/step - loss: 0.0066 - val_loss: 0.0012
Epoch 8/200
27/27 [==============================] - 0s 10ms/step - loss: 0.0050 - val_loss: 7.1199e-04
Epoch 9/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0041 - val_loss: 8.3050e-04
Epoch 10/200
27/27 [==============================] - 0s 8ms/step - loss: 0.0035 - val_loss: 6.